# Perfil de comércio

## Criar directorias de apoio 

In [2]:
from pathlib import Path

Path('support').mkdir(parents=True, exist_ok=True)
Path('reports').mkdir(parents=True, exist_ok=True)

## Criar ficheiro de configuração se não existir

O ficheiro de configuração serve para armazenar
a APIKEY que dá acesso a mais de 500 linhas de
resultados.


In [3]:
import os

fname = 'config.ini'
content = """
# Ficheiro de configuração
[comtrade]
# Add API Key. DO NOT SHARE
key = 
"""
if not os.path.isfile(fname):
  print("Criando ficheiro de configuração")
  with open(fname,'w') as f:
    f.write(content)


## Obter uma chave de acesso à API

Para aceder à UN Comtrade via API sem limites é necessário uma chave de acesso,
de outro modo os resultados são limitados a 500 linhas.

Para obter a chave de acsso:
* Registo em https://comtradedeveloper.un.org/
* Ir para _Products_ 
* Selecionar "Premium Individual APIs" (https://comtradedeveloper.un.org/product#product=dataapis)
* Escolher _Subscribe to "comtrade - v1"_  
* Esperar pelo email com a chave da API key (demora alguns dias)
* Copiar a chave para o local indicado no ficheiro `config.ini` antes
  de executar o resto do notebook.


## Inicializar a API e transferir os códigos usados

In [4]:
import os
import configparser
import comtrade

if os.path.isfile(fname):

    config = configparser.ConfigParser()
    config.read('config.ini')
    # get API Key or set to None    
    APIKEY = config['comtrade'].get('key', None)

comtrade.init(APIKEY, force_init=True)
comtrade.PERIOD_SECONDS=5


# Trade profile

Choose country and years of interest

In [5]:
country_of_interest = comtrade.m49_angola
years_of_interest = comtrade.year_range(2021, 2022)   
print(comtrade.COUNTRY_CODES[country_of_interest], years_of_interest)

Angola 2021


### Global exports and imports

In [6]:
cols = ['reporterDesc','partnerDesc','flowDesc','primaryValueFormated']
global_trade = comtrade.get_global_stats(country_of_interest, years_of_interest)
global_trade[cols].sort_index()


reporterDesc partnerDesc flowDesc primaryValueFormated
refYear flowCode                                                       
2021    M              Angola       World   Import    11,378,927,920.70
        X              Angola       World   Export    34,472,161,956.83

## Principais parceiros comerciais
### Principais destinos das exportações

In [7]:
show_ony_top=20
top_partners = comtrade.top_partners(country_of_interest,years_of_interest,flowCode='X', 
                                    rank_filter=show_ony_top)

In [8]:
cols=['reporterDesc','refYear','flowDesc','rank_partner','partnerDesc','primaryValueFormated']
top_partners[cols].sort_values(by=['refYear','flowDesc','rank_partner'])

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,primaryValueFormated
348,Angola,2021,Export,1,China,"20,058,426,271.91"
211,Angola,2021,Export,2,India,"3,008,390,595.07"
288,Angola,2021,Export,3,United Arab Emirates,"1,399,449,578.20"
165,Angola,2021,Export,4,Thailand,"1,004,833,529.77"
46,Angola,2021,Export,5,Singapore,"802,659,408.76"
109,Angola,2021,Export,6,Spain,"793,986,998.99"
67,Angola,2021,Export,7,Italy,"632,025,040.54"
279,Angola,2021,Export,8,South Africa,"519,358,721.33"
70,Angola,2021,Export,9,Chile,"518,264,559.23"
63,Angola,2021,Export,10,Netherlands,"477,332,019.79"


# Exportações

## Principais produtos exportados

In [9]:
show_only_top=20
export_products = comtrade.top_commodities(country_of_interest,
                                               partnerCode=0,  # 0 = all partners
                                               years=years_of_interest, 
                                               cmdCode='AG2',
                                               flowCode='X',
                                           motCode=0,
                                           rank_filter=show_only_top)


In [10]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdCode','cmdDesc', 'primaryValueFormated',
        comtrade.PERC_CMD_IN_PARTNER]
export_products[cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdCode,cmdDesc,primaryValueFormated,perc_cmd_for_partner
279,Angola,2021,Export,1,27,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","31,318,918,599.48",90.85%
83,Angola,2021,Export,2,71,"Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with precious metal, and articles thereof; imitation jewellery; coin","1,537,399,842.01",4.46%
201,Angola,2021,Export,3,89,"Ships, boats and floating structures","1,166,123,104.13",3.38%
287,Angola,2021,Export,4,84,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof","84,258,684.96",0.24%
178,Angola,2021,Export,5,44,Wood and articles of wood; wood charcoal,"49,678,464.60",0.14%
270,Angola,2021,Export,6,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement","46,892,142.38",0.14%
235,Angola,2021,Export,7,03,"Fish and crustaceans, molluscs and other aquatic invertebrates","43,903,585.95",0.13%
92,Angola,2021,Export,8,90,"Optical, photographic, cinematographic, measuring, checking, medical or surgical instruments and apparatus; parts and accessories","40,063,322.62",0.12%
210,Angola,2021,Export,9,73,Iron or steel articles,"23,234,088.08",0.07%
90,Angola,2021,Export,10,22,"Beverages, spirits and vinegar","22,483,179.13",0.07%


## Principais produtos exportados e destinos

In [11]:
show_only_top = 5
export_products = comtrade.top_commodities(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           flowCode='X',
                                           motCode=0, 
                                           extra_cols=[],
                                           rank_filter=show_only_top)

In [12]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdDesc','rank_partner','partnerDesc',
        comtrade.PERC_PARTNER_IN_CMD, 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER ]

show_only_greater_than = export_products[comtrade.PERC_PARTNER_IN_CMD]>0.01
export_products[show_only_greater_than][cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdDesc,rank_partner,partnerDesc,perc_partner_for_cmd,primaryValueFormated,perc_cmd_for_partner
2216,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",1,China,63.93%,"20,022,774,116.55",99.82%
4743,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",2,India,9.28%,"2,907,767,269.85",96.66%
5779,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",4,Thailand,3.21%,"1,004,404,810.25",99.96%
2222,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",5,Singapore,2.40%,"751,652,390.95",93.65%
4827,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",6,Spain,2.01%,"628,140,238.31",79.11%
1684,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",7,Italy,2.00%,"625,729,106.51",99.00%
5824,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",9,Chile,1.65%,"516,869,632.99",99.73%
2213,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",8,South Africa,1.56%,"489,628,802.00",94.28%
2343,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",10,Netherlands,1.46%,"456,193,575.29",95.57%
4506,Angola,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",12,Indonesia,1.42%,"443,927,852.71",99.93%


## Principais **parceiros** das exportações e produtos exportados

In [14]:
show_only_top = 5
export_partners = comtrade.top_partners(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           cmdCode='AG2',
                                           flowCode='X',
                                           motCode=0, 
                                           rank_filter=5)

In [15]:
cols = ['reporterDesc','refYear', 'flowDesc','rank_partner',	'partnerDesc','rank_cmd', 'cmdDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]
export_partners[cols].sort_values(['refYear','flowDesc','rank_partner']).style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, 
                                                                                       comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,rank_cmd,cmdDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
2216,Angola,2021,Export,1,China,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","20,022,774,116.55",99.82%,63.93%
5170,Angola,2021,Export,1,China,6,"Salt; sulphur; earths, stone; plastering materials, lime and cement","11,961,705.37",0.06%,25.51%
4696,Angola,2021,Export,1,China,5,Wood and articles of wood; wood charcoal,"11,645,905.65",0.06%,23.44%
890,Angola,2021,Export,1,China,13,"Ores, slag and ash","6,616,586.45",0.03%,66.65%
1100,Angola,2021,Export,1,China,18,Aluminium and articles thereof,"966,966.75",0.00%,17.60%
4743,Angola,2021,Export,2,India,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","2,907,767,269.85",96.66%,9.28%
3057,Angola,2021,Export,2,India,2,"Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with precious metal, and articles thereof; imitation jewellery; coin","100,217,986.40",3.33%,6.52%
1864,Angola,2021,Export,2,India,3,"Ships, boats and floating structures","106,755.20",0.00%,0.01%
1254,Angola,2021,Export,2,India,6,"Salt; sulphur; earths, stone; plastering materials, lime and cement","70,361.90",0.00%,0.15%
586,Angola,2021,Export,2,India,73,Pulp of wood or other fibrous cellulosic material; recovered (waste and scrap) paper or paperboard,"46,527.16",0.00%,99.32%


# Importações

## Principais produtos importados

In [19]:
show_only_top=10
export_products = comtrade.top_commodities(country_of_interest,
                                               partnerCode=0,  # 0 = all partners
                                               years=years_of_interest, 
                                               flowCode='M',
                                           motCode=0,
                                           rank_filter=10)


In [23]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdCode','cmdDesc', 'primaryValueFormated',
        'partnerDesc',
        comtrade.PERC_CMD_IN_PARTNER ]
export_products[cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdCode,cmdDesc,primaryValueFormated,partnerDesc,perc_cmd_for_partner
402,Angola,2021,Import,1,27,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","1,886,757,547.56",World,16.58%
230,Angola,2021,Import,2,84,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof","1,809,939,647.27",World,15.91%
126,Angola,2021,Import,3,87,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof","779,123,345.04",World,6.85%
367,Angola,2021,Import,4,85,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles","776,747,585.90",World,6.83%
161,Angola,2021,Import,5,10,Cereals,"584,957,170.07",World,5.14%
27,Angola,2021,Import,6,73,Iron or steel articles,"536,792,046.75",World,4.72%
362,Angola,2021,Import,7,39,Plastics and articles thereof,"405,040,625.98",World,3.56%
240,Angola,2021,Import,8,02,Meat and edible meat offal,"348,250,700.18",World,3.06%
322,Angola,2021,Import,9,30,Pharmaceutical products,"322,068,826.04",World,2.83%
169,Angola,2021,Import,10,15,"Animal, vegetable or microbial fats and oils and their cleavage products; prepared edible fats; animal or vegetable waxes","311,937,457.18",World,2.74%


## Principais produtos importados e origens

In [26]:
show_only_top = 10
export_products = comtrade.top_commodities(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           flowCode='M',
                                           cmdCode='AG2',
                                           motCode=0, 
                                           rank_filter=show_only_top)

In [27]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdDesc','rank_partner','partnerDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]

filter_relevant_partners = export_products[comtrade.PERC_PARTNER_IN_CMD] > 0.05  # show only partners with more than 5% of trade
export_products[filter_relevant_partners][cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdDesc,rank_partner,partnerDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
40394,Angola,2021,Import,1,"Petroleum oils and oils from bituminous minerals, not crude; preparations n.e.c, containing by weight 70% or more of petroleum oils or oils from bituminous minerals; these being the basic constituents of the preparations; waste oils",4,Togo,"687,679,076.53",100.00%,36.91%
44235,Angola,2021,Import,1,"Petroleum oils and oils from bituminous minerals, not crude; preparations n.e.c, containing by weight 70% or more of petroleum oils or oils from bituminous minerals; these being the basic constituents of the preparations; waste oils",8,Netherlands,"318,086,314.98",73.61%,17.07%
37681,Angola,2021,Import,1,"Petroleum oils and oils from bituminous minerals, not crude; preparations n.e.c, containing by weight 70% or more of petroleum oils or oils from bituminous minerals; these being the basic constituents of the preparations; waste oils",3,India,"290,323,266.21",38.93%,15.58%
37192,Angola,2021,Import,1,"Petroleum oils and oils from bituminous minerals, not crude; preparations n.e.c, containing by weight 70% or more of petroleum oils or oils from bituminous minerals; these being the basic constituents of the preparations; waste oils",17,Malta,"184,307,230.08",99.00%,9.89%
17368,Angola,2021,Import,1,"Petroleum oils and oils from bituminous minerals, not crude; preparations n.e.c, containing by weight 70% or more of petroleum oils or oils from bituminous minerals; these being the basic constituents of the preparations; waste oils",11,Belgium,"113,667,644.69",29.22%,6.10%
9543,Angola,2021,Import,2,Wheat and meslin,10,France and Monaco,"126,789,179.29",30.82%,44.87%
10167,Angola,2021,Import,2,Wheat and meslin,19,Russian Federation,"68,553,827.63",37.07%,24.26%
17830,Angola,2021,Import,2,Wheat and meslin,25,Argentina,"39,006,543.78",32.19%,13.81%
37519,Angola,2021,Import,2,Wheat and meslin,39,Lithuania,"17,400,726.00",83.91%,6.16%
15194,Angola,2021,Import,2,Wheat and meslin,18,Germany,"14,371,958.18",7.75%,5.09%


### Principal origem das importações

In [28]:
show_ony_top=10
top_partners = comtrade.top_partners(country_of_interest,years_of_interest,
                                     flowCode='M', 
                                    rank_filter=show_ony_top)

In [28]:
cols=['reporterDesc','refYear','flowDesc','rank_partner','partnerDesc','primaryValueFormated']
top_partners[cols].sort_values(by=['refYear','flowDesc','rank_partner'])

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,primaryValueFormated
258,Angola,2021,Import,1,China,"1,688,904,641.34"
95,Angola,2021,Import,2,Portugal,"1,357,979,633.71"
323,Angola,2021,Import,3,India,"745,723,527.22"
319,Angola,2021,Import,4,Togo,"687,689,058.08"
495,Angola,2021,Import,5,Brazil,"546,044,463.65"
610,Angola,2021,Import,6,United Kingdom,"539,546,476.65"
251,Angola,2021,Import,7,United States of America,"523,006,089.83"
455,Angola,2021,Import,8,Netherlands,"432,100,821.51"
192,Angola,2021,Import,9,Italy,"427,046,959.44"
36,Angola,2021,Import,10,France and Monaco,"411,360,213.52"


## Principais **parceiros** das importações e produtos importados

In [31]:
show_only_top = 10
export_partners = comtrade.top_partners(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           cmdCode='AG2',
                                           flowCode='M',
                                           motCode=0, 
                                           rank_filter=show_only_top)

In [32]:
cols = ['reporterDesc','refYear', 'flowDesc','rank_partner','partnerDesc', 'cmdDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]

filter_relevant_partners = export_partners[comtrade.PERC_PARTNER_IN_CMD] > 0.0  # show only partners with more than 5% of trade
export_partners[filter_relevant_partners][cols].sort_values(['refYear','flowDesc','rank_partner']).style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,cmdDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
7421,Angola,2021,Import,1,China,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof","294,737,063.36",17.45%,16.28%
1837,Angola,2021,Import,1,China,"Textiles, made up articles; sets; worn clothing and worn textile articles; rags","207,543,201.07",12.29%,80.04%
5764,Angola,2021,Import,1,China,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles","188,027,402.48",11.13%,24.21%
3683,Angola,2021,Import,1,China,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof","170,241,136.72",10.08%,21.85%
12591,Angola,2021,Import,1,China,Iron and steel,"94,179,850.47",5.58%,64.83%
5371,Angola,2021,Import,1,China,Plastics and articles thereof,"90,509,512.27",5.36%,22.35%
11724,Angola,2021,Import,1,China,Glass and glassware,"62,248,517.42",3.69%,75.37%
6889,Angola,2021,Import,1,China,Iron or steel articles,"60,344,496.23",3.57%,11.24%
6008,Angola,2021,Import,1,China,Rubber and articles thereof,"58,498,228.59",3.46%,43.76%
11490,Angola,2021,Import,1,China,Pharmaceutical products,"48,373,928.61",2.86%,15.02%


## Análise de simetrias nos dados

In [33]:


partnerA = country_of_interest


### Principais destinos de exportação reportados pelo país

In [34]:
show_only_top = 10
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode='X',
                     cmdCode='TOTAL',
                     reporterCode=partnerA,
                     partnerCode=None,
                     partner2Code=0,
                     period=years_of_interest,
                     motCode = 0,
                     customsCode=None,
                     echo_url=False
                     )
cols = ['reporterDesc','partnerDesc','partner2Desc','refYear','flowDesc','primaryValueFormated','primaryValue','perc','motDesc','isAggregate','customsDesc'] 
total_import = df_import_rep_a_world['primaryValue'].sum()
df_import_rep_a_world['perc'] = df_import_rep_a_world['primaryValue']/total_import
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>100000][cols].sort_values(by=['refYear','primaryValue'], ascending=[True,False]).head(show_only_top).style.format({'perc': '{:,.2%}'.format}) 


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
348,Angola,China,World,2021,Export,"20,058,426,271.91",20058426271.912998,58.19%,All modes of transport,True,All customs procedure codes
211,Angola,India,World,2021,Export,"3,008,390,595.07",3008390595.072000,8.73%,All modes of transport,True,All customs procedure codes
288,Angola,United Arab Emirates,World,2021,Export,"1,399,449,578.20",1399449578.198000,4.06%,All modes of transport,True,All customs procedure codes
165,Angola,Thailand,World,2021,Export,"1,004,833,529.77",1004833529.770000,2.91%,All modes of transport,True,All customs procedure codes
46,Angola,Singapore,World,2021,Export,"802,659,408.76",802659408.761000,2.33%,All modes of transport,True,All customs procedure codes
109,Angola,Spain,World,2021,Export,"793,986,998.99",793986998.994000,2.30%,All modes of transport,True,All customs procedure codes
67,Angola,Italy,World,2021,Export,"632,025,040.54",632025040.539000,1.83%,All modes of transport,True,All customs procedure codes
279,Angola,South Africa,World,2021,Export,"519,358,721.33",519358721.333000,1.51%,All modes of transport,True,All customs procedure codes
70,Angola,Chile,World,2021,Export,"518,264,559.23",518264559.233000,1.50%,All modes of transport,True,All customs procedure codes
63,Angola,Netherlands,World,2021,Export,"477,332,019.79",477332019.785000,1.38%,All modes of transport,True,All customs procedure codes


### Principais destinos de exportação segundo o registo da importações do resto do mundo

In [35]:
show_only_top=10
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode='M',
                     cmdCode='TOTAL',
                     reporterCode=None,
                     partnerCode=partnerA,
                     partner2Code=0,
                     motCode = 0,
                     # customsCode=0,
                     period=years_of_interest,
                     echo_url=False
                     )
total_import = df_import_rep_a_world['primaryValue'].sum()
df_import_rep_a_world['perc'] = df_import_rep_a_world['primaryValue']/total_import
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>1000][cols].sort_values(by=['refYear','primaryValue','reporterDesc',], ascending=[True,False,True]).head(show_only_top).style.format({'perc': '{:,.2%}'.format}) 

,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
29,China,Angola,World,2021,Import,"20,908,231,317.00",20908231317.000000,62.45%,All modes of transport,True,All customs procedure codes
324,India,Angola,World,2021,Import,"2,315,947,568.41",2315947568.408000,6.92%,All modes of transport,True,All customs procedure codes
388,United Arab Emirates,Angola,World,2021,Import,"1,685,701,656.52",1685701656.518000,5.04%,All modes of transport,True,All customs procedure codes
385,Thailand,Angola,World,2021,Import,"1,607,756,231.51",1607756231.510000,4.80%,All modes of transport,True,All customs procedure codes
413,United States of America,Angola,World,2021,Import,"1,089,250,402.00",1089250402.000000,3.25%,All modes of transport,True,All customs procedure codes
325,Singapore,Angola,World,2021,Import,"752,550,761.40",752550761.398000,2.25%,All modes of transport,True,All customs procedure codes
355,Spain,Angola,World,2021,Import,"739,484,782.77",739484782.768000,2.21%,All modes of transport,True,All customs procedure codes
5,Belgium,Angola,World,2021,Import,"615,024,126.87",615024126.871000,1.84%,All modes of transport,True,All customs procedure codes
28,Chile,Angola,World,2021,Import,"537,823,552.96",537823552.960000,1.61%,All modes of transport,True,All customs procedure codes
147,Indonesia,Angola,World,2021,Import,"463,466,072.00",463466072.000000,1.38%,All modes of transport,True,All customs procedure codes


## Análise de simetrias nos dados com um país específico

In [36]:
partnerB = comtrade.COUNTRY_CODES_REVERSE['United Arab Emirates']
flowCode = 'X'
cmd_detail = 'AG4'
show_only_top = 10

flowDesc = comtrade.FLOWS_CODES[flowCode]
cols = ['reporterDesc','partnerDesc','partner2Desc','refYear','flowDesc','cmdDesc','primaryValueFormated','primaryValue','perc','motDesc','isAggregate','customsDesc'] 

In [37]:
df_import_rep_a = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode=cmd_detail,
                     reporterCode=partnerA,
                     partnerCode=partnerB,
                     partner2Code=0,
                     period=2020,
                     motCode = 0,
                     echo_url=False
                     )


In [38]:
if df_import_rep_a is not None:
    total_import = df_import_rep_a['primaryValue'].sum()
    df_import_rep_a['perc'] = df_import_rep_a['primaryValue']/total_import
    cols = cols + ['perc']
    print(f"'{flowDesc}' de produtos de {comtrade.COUNTRY_CODES.get(partnerB)} por {comtrade.COUNTRY_CODES.get(partnerA)} reportados por {comtrade.COUNTRY_CODES.get(partnerA)}")
    result = df_import_rep_a[cols].sort_values(by=['refYear','primaryValue'], ascending=[True,False]).head(show_only_top).style.format({'perc': '{:,.2%}'.format})
else:
    print(f"Não há '{flowDesc}' de produtos de {comtrade.COUNTRY_CODES.get(partnerA)} por {comtrade.COUNTRY_CODES.get(partnerB)} reportados por {comtrade.COUNTRY_CODES.get(partnerA)}")
    result = ''

result

'Export' de produtos de United Arab Emirates por Angola reportados por Angola


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,cmdDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc,perc
42,Angola,United Arab Emirates,World,2020,Export,"Diamonds, whether or not worked, but not mounted or set","896,305,977.53",896305977.527000,85.44%,All modes of transport,True,All customs procedure codes,85.44%
204,Angola,United Arab Emirates,World,2020,Export,"Light-vessels, fire-floats, dredgers, floating cranes, other vessels; the navigability of which is subsidiary to main function; floating docks, floating, submersible drilling, production platforms","96,461,019.73",96461019.730000,9.20%,All modes of transport,True,All customs procedure codes,9.20%
6,Angola,United Arab Emirates,World,2020,Export,Petroleum gases and other gaseous hydrocarbons,"24,056,888.36",24056888.356000,2.29%,All modes of transport,True,All customs procedure codes,2.29%
4,Angola,United Arab Emirates,World,2020,Export,Petroleum oils and oils obtained from bituminous minerals; crude,"17,215,786.33",17215786.326000,1.64%,All modes of transport,True,All customs procedure codes,1.64%
206,Angola,United Arab Emirates,World,2020,Export,"Vessels; other, including warships and lifeboats, other than rowing boats","8,098,008.10",8098008.099000,0.77%,All modes of transport,True,All customs procedure codes,0.77%
87,Angola,United Arab Emirates,World,2020,Export,Aluminium; articles n.e.c. in chapter 76,"1,404,485.22",1404485.215000,0.13%,All modes of transport,True,All customs procedure codes,0.13%
197,Angola,United Arab Emirates,World,2020,Export,"Motor vehicles; parts and accessories, of heading no. 8701 to 8705","817,712.48",817712.478000,0.08%,All modes of transport,True,All customs procedure codes,0.08%
79,Angola,United Arab Emirates,World,2020,Export,Copper tubes and pipes,"676,801.14",676801.138000,0.06%,All modes of transport,True,All customs procedure codes,0.06%
61,Angola,United Arab Emirates,World,2020,Export,"Nails, tacks, drawing pins, corrugated nails, staples (not those of heading no. 8305) and the like, of iron or steel, with heads of other material or not, but excluding articles with heads of copper","536,615.99",536615.986000,0.05%,All modes of transport,True,All customs procedure codes,0.05%
202,Angola,United Arab Emirates,World,2020,Export,"Cruise ships, excursion boats, ferry-boats, cargo ships, barges and similar vessels for the transport of persons or goods","341,281.47",341281.467000,0.03%,All modes of transport,True,All customs procedure codes,0.03%


In [55]:
# invertemos

print(years_of_interest, partnerA, partnerB)

if flowCode == 'M':
    flowCodeInv = 'X'
else:
    flowCodeInv = 'M'

flowDesc = comtrade.FLOWS_CODES[flowCodeInv]
print(f"'{flowDesc}' de produtos de {comtrade.COUNTRY_CODES.get(partnerB)} para {comtrade.COUNTRY_CODES.get(partnerA)} reportados por {comtrade.COUNTRY_CODES.get(partnerB)}")

df_import_rep_b = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCodeInv,
                     cmdCode=cmd_detail,
                     reporterCode=partnerB,
                     partnerCode=partnerA,
                     partner2Code=0,
                     period=years_of_interest,
                     motCode = 0,
                     echo_url=False
                     )
if df_import_rep_b is not None:
    total_import = df_import_rep_b['primaryValue'].sum()
    df_import_rep_b['perc'] = df_import_rep_b['primaryValue']/total_import
    cols = cols + ['perc']
    result = df_import_rep_b[cols].sort_values(by=['refYear','reporterDesc','primaryValue'], ascending=[True,True,False]).head(show_only_top).style.format({'perc': '{:,.2%}'.format})
else:
    print(f"Não há '{flowDesc}' de produtos de {comtrade.COUNTRY_CODES.get(partnerB)} para {comtrade.COUNTRY_CODES.get(partnerA)} reportados por {comtrade.COUNTRY_CODES.get(partnerB)}")
    result = ''
result

2021 24 784
'Import' de produtos de United Arab Emirates para Angola reportados por United Arab Emirates


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,cmdDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc,perc,perc
28,United Arab Emirates,Angola,World,2021,Import,"Diamonds, whether or not worked, but not mounted or set","1,642,061,333.94",1642061333.941000,97.41%,All modes of transport,True,All customs procedure codes,97.41%,97.41%
29,United Arab Emirates,Angola,World,2021,Import,"Gold (including gold plated with platinum) unwrought or in semi-manufactured forms, or in powder form","41,082,546.98",41082546.981000,2.44%,All modes of transport,True,All customs procedure codes,2.44%,2.44%
35,United Arab Emirates,Angola,World,2021,Import,Aluminium; waste and scrap,"541,582.04",541582.041000,0.03%,All modes of transport,True,All customs procedure codes,0.03%,0.03%
21,United Arab Emirates,Angola,World,2021,Import,"Bed linen, table linen, toilet linen and kitchen linen","352,533.69",352533.687000,0.02%,All modes of transport,True,All customs procedure codes,0.02%,0.02%
7,United Arab Emirates,Angola,World,2021,Import,"Trunks; suit, camera, jewellery, cutlery cases; travel, tool, similar bags; wholly or mainly covered by leather, composition leather, plastic sheeting, textile materials, vulcanised fibre, paperboard","308,440.20",308440.196000,0.02%,All modes of transport,True,All customs procedure codes,0.02%,0.02%
50,United Arab Emirates,Angola,World,2021,Import,"Paintings, drawings and pastels, executed entirely by hand; not drawings of heading no. 4906 and not hand-painted, hand-decorated manufactured articles; collages, mosaics and similar decorative plaques","296,321.30",296321.299000,0.02%,All modes of transport,True,All customs procedure codes,0.02%,0.02%
9,United Arab Emirates,Angola,World,2021,Import,"Wood in the rough, whether or not stripped of bark or sapwood, or roughly squared","273,812.65",273812.654000,0.02%,All modes of transport,True,All customs procedure codes,0.02%,0.02%
10,United Arab Emirates,Angola,World,2021,Import,"Wood sawn or chipped lengthwise, sliced or peeled, whether or not planed, sanded or finger-jointed, of a thickness exceeding 6mm","210,078.33",210078.333000,0.01%,All modes of transport,True,All customs procedure codes,0.01%,0.01%
34,United Arab Emirates,Angola,World,2021,Import,Copper wire,"184,783.93",184783.929000,0.01%,All modes of transport,True,All customs procedure codes,0.01%,0.01%
51,United Arab Emirates,Angola,World,2021,Import,"Sculptures and statuary; original, in any material","159,394.96",159394.958000,0.01%,All modes of transport,True,All customs procedure codes,0.01%,0.01%
